# Google Agent Development Kit (ADK) Tutorial

**A hands-on guide to building AI agents with Google's ADK**

---

## Table of Contents

1. [Why ADK?](#1-why-adk) — What ADK is and why you'd use it
2. [Setup](#2-setup) — Installation and project structure
3. [Hello World Agent](#3-hello-world-agent) — Your first agent
4. [CLI Interaction](#4-cli-interaction) — `adk run`, `adk web`, and Runner API
5. [Agent Types](#5-agent-types) — Sequential, Parallel, and Loop agents
6. [Tools](#6-tools) — Function tools, Google Search, BigQuery
7. [Callbacks](#7-callbacks) — Guardrails, logging, and lifecycle hooks
8. [Sessions, State, and Memory](#8-sessions-state-and-memory) — Context management
9. [Clean Up](#9-clean-up) — Remove generated files

### Prerequisites

- Python 3.10+
- Google Cloud project with Vertex AI enabled
- `gcloud auth application-default login` completed
- Basic Python and async/await familiarity

### Learning Objectives

By the end of this notebook, you will be able to:

- Explain what ADK is and how it compares to alternatives
- Create and run ADK agents with tools
- Compose agents using Sequential, Parallel, and Loop patterns
- Use built-in tools like Google Search and BigQuery
- Implement callbacks for guardrails and logging
- Manage sessions, state, and memory across interactions

---

## 1. Why ADK?

### What is the Agent Development Kit?

Google's **Agent Development Kit (ADK)** is an open-source, code-first framework for building, evaluating, and deploying AI agents. Announced at Google Cloud NEXT 2025, it's the same framework that powers Google's own products like Agentspace and Customer Engagement Suite.

**Core philosophy:** Agent development should feel like software development — version control, testing, modularity, and code review all apply.

**Key features:**

- **Code-first**: Agents are defined in Python (also TypeScript, Go, Java) — not dragged and dropped
- **Model-agnostic**: Optimized for Gemini, but supports 100+ LLMs via LiteLLM (Claude, GPT-4, Llama, Mistral, etc.)
- **Deployment-agnostic**: Run locally, on Cloud Run, GKE, Vertex AI Agent Engine, or any container host
- **Full lifecycle**: Build → Interact → Evaluate → Deploy, all within one framework

### ADK vs. DIY

![Why do you need a framework?](images/image(4).png)

| Layer | DIY (Build Yourself) | ADK (Out of the Box) |
|-------|---------------------|---------------------|
| **LLM Integration** | Custom LLM wrapper | `Agent` + `LlmAgent` — model-agnostic |
| **Tool Execution** | Custom tool engine | `FunctionTool` · `MCP` · `OpenAPI` — auto schema generation |
| **Multi-Agent Orchestration** | Custom orchestration | `Sequential` · `Parallel` · `Loop` workflow agents |
| **Session & State** | Custom session management | `Session` · `State` · `Memory` — multiple backends |
| **Evaluation** | Custom eval framework | `adk eval` — built-in metrics |
| **Deployment** | Custom infra | Cloud Run · GKE · Agent Engine — one-command deploy |
| **Observability** | Custom logging | Cloud Trace · Monitoring — OpenTelemetry built-in |

Building from scratch means implementing every layer yourself. ADK provides all of these out of the box.

### ADK Architecture Overview

![ADK Architecture](images/image(6).png)

The ADK SDK sits between you (the developer) and your end users. It provides CLI tools for development, connects to AI models, and deploys to multiple targets — all from the same codebase.

---

## 2. Setup

### Installing ADK

ADK requires Python 3.10 or later. Install it with pip:

In [79]:
!pip install google-adk

In [80]:
# Verify the installation
!pip show google-adk

Name: google-adk
Version: 1.25.0
Summary: Agent Development Kit
Home-page: https://google.github.io/adk-docs/
Author: 
Author-email: Google LLC <googleapis-packages@google.com>
License: 
Location: /home/admin_/projects/adk-learning/.venv/lib/python3.12/site-packages
Requires: aiosqlite, anyio, authlib, click, fastapi, google-api-python-client, google-auth, google-cloud-aiplatform, google-cloud-bigquery, google-cloud-bigquery-storage, google-cloud-bigtable, google-cloud-discoveryengine, google-cloud-pubsub, google-cloud-secret-manager, google-cloud-spanner, google-cloud-speech, google-cloud-storage, google-genai, graphviz, httpx, jsonschema, mcp, opentelemetry-api, opentelemetry-exporter-gcp-logging, opentelemetry-exporter-gcp-monitoring, opentelemetry-exporter-gcp-trace, opentelemetry-exporter-otlp-proto-http, opentelemetry-resourcedetector-gcp, opentelemetry-sdk, pyarrow, pydantic, python-dateutil, python-dotenv, PyYAML, requests, sqlalchemy, sqlalchemy-spanner, starlette, tenacity, t

### ADK Project Structure

![Project Structure](images/image(8).png)

Every ADK agent lives in a **directory** with three files:

| File | Purpose |
|------|--------|
| `__init__.py` | Imports the agent module (`from . import agent`) |
| `agent.py` | Defines `root_agent` — the entry point ADK discovers |
| `.env` | API keys and project configuration |

The variable **must** be named `root_agent` — this is how ADK's CLI tools find your agent.

### Environment Configuration

ADK supports two backends for model access:

**Vertex AI** (recommended for GCP users):
```
GOOGLE_GENAI_USE_VERTEXAI=TRUE
GOOGLE_CLOUD_PROJECT=agentspace-testing-471714
GOOGLE_CLOUD_LOCATION=us-central1
```

Requires: `gcloud auth application-default login`

**Google AI Studio** (simpler, API key only):
```
GOOGLE_GENAI_USE_VERTEXAI=FALSE
GOOGLE_API_KEY=your-api-key
```

This tutorial uses **Vertex AI**. Make sure your project has the Vertex AI API enabled.

---

## 3. Hello World Agent

Let's build our first agent. We'll create the standard ADK project structure using `%%writefile` magic so everything stays self-contained in this notebook.

In [81]:
import os

# Set environment variables for Vertex AI — required for programmatic Runner usage.
# The .env files are only auto-loaded by CLI tools (adk run, adk web).
os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "TRUE"
os.environ["GOOGLE_CLOUD_PROJECT"] = "agentspace-testing-471714"
os.environ["GOOGLE_CLOUD_LOCATION"] = "us-central1"

os.makedirs("hello_agent", exist_ok=True)

In [82]:
%%writefile hello_agent/__init__.py
from . import agent

Overwriting hello_agent/__init__.py


In [83]:
%%writefile hello_agent/.env
GOOGLE_GENAI_USE_VERTEXAI=TRUE
GOOGLE_CLOUD_PROJECT=agentspace-testing-471714
GOOGLE_CLOUD_LOCATION=us-central1

Overwriting hello_agent/.env


In [84]:
%%writefile hello_agent/agent.py
from google.adk.agents import Agent

root_agent = Agent(
    name="hello_agent",
    model="gemini-2.5-flash",
    description="A simple greeting agent.",
    instruction="You are a friendly assistant. Greet the user and answer their questions concisely.",
)

Overwriting hello_agent/agent.py


In [85]:
# Run the agent programmatically using Runner + InMemorySessionService
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai.types import Content, Part
from hello_agent.agent import root_agent

# Create the session service and runner
session_service = InMemorySessionService()
runner = Runner(
    agent=root_agent,
    app_name="hello_app",
    session_service=session_service,
)

# Create a session
session = await session_service.create_session(
    app_name="hello_app",
    user_id="tutorial_user",
)

# Send a message and collect the response
message = Content(parts=[Part(text="Hello! What is Google's Agent Development Kit?")], role="user")

response_text = ""
async for event in runner.run_async(
    user_id="tutorial_user",
    session_id=session.id,
    new_message=message,
):
    if event.is_final_response() and event.content and event.content.parts:
        response_text = event.content.parts[0].text

print(response_text)

Hi there!

Google's Agent Development Kit (ADK) is a framework that helps developers create conversational AI experiences, especially for devices and services integrated with Google Assistant. It provides tools and resources to build and deploy agents that can understand natural language and perform actions.


### Code Walkthrough

Let's break down what just happened:

1. **`Runner`** — The central orchestrator. It manages the event loop, coordinates the agent, LLM, tools, and services.

2. **`InMemorySessionService`** — Stores session data in memory. Fine for development; use `DatabaseSessionService` or `VertexAiSessionService` for production.

3. **`Content` and `Part`** — ADK's message format. `Content` is a container with a `role` ("user" or "model") and `Part` objects hold the actual data (text, function calls, etc.).

4. **`runner.run_async()`** — Returns an async generator of `Event` objects. Each event represents something that happened during execution (LLM responses, tool calls, state changes). We filter for `is_final_response()` to get the agent's answer.

5. **`Event.is_final_response()`** — Returns `True` when the event contains the agent's final text output (no pending tool calls or streaming chunks).

In [86]:
# Multi-turn conversation — the agent remembers context within the same session
follow_up = Content(parts=[Part(text="Can you give me a one-sentence summary of what you just said?")], role="user")

response_text = ""
async for event in runner.run_async(
    user_id="tutorial_user",
    session_id=session.id,  # Same session — context is preserved
    new_message=follow_up,
):
    if event.is_final_response() and event.content and event.content.parts:
        response_text = event.content.parts[0].text

print(response_text)

Google's Agent Development Kit (ADK) is a framework that enables developers to build and deploy conversational AI agents for Google Assistant, allowing them to understand natural language and perform actions.


---

## 4. CLI Interaction

ADK provides three ways to interact with your agent:

| Mode | Command | Interface | Key Features |
|------|---------|-----------|-------------|
| **Terminal Chat** | `adk run` | CLI | Interactive or piped input; session save/resume |
| **Browser Dev UI** | `adk web` | Browser | Chat + state inspector; events, traces, eval tabs |
| **Programmatic** | `Runner` API | Python | `runner.run_async()`; full control in notebooks & apps |

In [87]:
# adk run with piped input (non-interactive, works in notebooks)
!echo "What is 2 + 2?" | adk run hello_agent

Log setup complete: /tmp/agents_log/agent.20260217_201254.log
To access latest log: tail -F /tmp/agents_log/agent.latest.log
/home/admin_/projects/adk-learning/.venv/lib/python3.12/site-packages/google/adk/cli/cli.py:189: UserWarning: [EXPERIMENTAL] InMemoryCredentialService: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  credential_service = InMemoryCredentialService()
/home/admin_/projects/adk-learning/.venv/lib/python3.12/site-packages/google/adk/auth/credential_service/in_memory_credential_service.py:33: UserWarning: [EXPERIMENTAL] BaseCredentialService: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  super().__init__()
Running agent hello_agent, type exit to exit.
[user]: [hello_agent]: Hello! 2 + 2 equals 4.
[user]: 
Aborted!


In [88]:
import subprocess
import time

# Launch adk web as a background process
adk_web_process = subprocess.Popen(
    [
        "adk", "web", ".",
        "--port", "8080",
        "--session_service_uri", "memory://",
    ],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
)

# Give the server a moment to start
time.sleep(3)

if adk_web_process.poll() is None:
    print("adk web server is running!")
    print("Open the dev UI at: http://localhost:8080")
else:
    print("Failed to start adk web server.")
    _, stderr = adk_web_process.communicate()
    print(f"stderr: {stderr.decode()}")

adk web server is running!
Open the dev UI at: http://localhost:8080


In [89]:
## Stop the adk web server
#if adk_web_process.poll() is None:
#    adk_web_process.terminate()
#    adk_web_process.wait(timeout=5)
#    print("adk web server stopped.")
#else:
#    print("adk web server was not running.")

### `adk api_server` — REST API

`adk api_server` starts a FastAPI server with Swagger docs at `/docs`. Key endpoints:

| Method | Path | Description |
|--------|------|-------------|
| `GET` | `/list-apps` | List available agents |
| `POST` | `/run` | Run agent (single JSON response) |
| `POST` | `/run_sse` | Run agent (Server-Sent Events stream) |

This is useful for integrating agents into web applications or microservices.

---

## 5. Agent Types

We already built an `LlmAgent` (aliased as `Agent`) in Section 3. Now let's look at the **workflow agents** that provide deterministic orchestration without LLM routing decisions.

![The Core of ADK — The Agent](images/image(1).png)

| Agent Class | Parent | Description | Key Features |
|------------|--------|-------------|-------------|
| **`LlmAgent`** (`Agent`) | `BaseAgent` | LLM-powered reasoning | Dynamic tool use, sub-agent delegation |
| **`SequentialAgent`** | `BaseAgent` | Executes sub-agents in order | Shared state via `output_key`, template variable passing |
| **`ParallelAgent`** | `BaseAgent` | Concurrent execution | Independent conversation history, shared `session.state` |
| **`LoopAgent`** | `BaseAgent` | Iterative refinement | `max_iterations` limit, `escalate` to exit |
| Custom Agents | `BaseAgent` | Extend `BaseAgent` | Whatever you need |

### SequentialAgent

Executes sub-agents **in order**, one after another. Data flows between steps via `output_key` and template variables:

1. Agent A runs, its response is saved to `session.state["result_a"]` via `output_key="result_a"`
2. Agent B's instruction references `{result_a}` — ADK resolves this from session state before sending to the LLM

```python
pipeline = SequentialAgent(
    name="Pipeline",
    sub_agents=[agent_a, agent_b, agent_c],
)
```

### ParallelAgent

Executes all sub-agents **concurrently**. Each branch has independent conversation history, but they share `session.state`. Event ordering may be non-deterministic.

```python
parallel = ParallelAgent(
    name="ParallelResearch",
    sub_agents=[researcher_a, researcher_b],
)
```

### LoopAgent

Repeatedly executes sub-agents in sequence until a termination condition is met:

1. `max_iterations` parameter (hard limit)
2. A sub-agent calls `tool_context.actions.escalate = True` to break out

```python
loop = LoopAgent(
    name="RefinementLoop",
    sub_agents=[critic, refiner],
    max_iterations=5,
)
```

In [90]:
import os
os.makedirs("workflow_agent", exist_ok=True)

In [91]:
%%writefile workflow_agent/__init__.py
from . import agent

Overwriting workflow_agent/__init__.py


In [92]:
%%writefile workflow_agent/.env
GOOGLE_GENAI_USE_VERTEXAI=TRUE
GOOGLE_CLOUD_PROJECT=agentspace-testing-471714
GOOGLE_CLOUD_LOCATION=us-central1

Overwriting workflow_agent/.env


In [93]:
%%writefile workflow_agent/agent.py
from google.adk.agents import Agent, SequentialAgent, ParallelAgent

# Two researchers that run concurrently
researcher_a = Agent(
    name="RenewableResearcher",
    model="gemini-2.5-flash",
    instruction="Write a short 2-3 sentence overview of recent advances in renewable energy.",
    output_key="renewable_result",
)

researcher_b = Agent(
    name="EVResearcher",
    model="gemini-2.5-flash",
    instruction="Write a short 2-3 sentence overview of recent advances in electric vehicles.",
    output_key="ev_result",
)

# Synthesizer reads both results via template variables
synthesizer = Agent(
    name="Synthesizer",
    model="gemini-2.5-flash",
    instruction=(
        "Combine these two research summaries into a single coherent paragraph "
        "about the intersection of clean energy and transportation:\n\n"
        "Renewable Energy: {renewable_result}\n\n"
        "Electric Vehicles: {ev_result}"
    ),
)

parallel_research = ParallelAgent(
    name="ParallelResearch",
    sub_agents=[researcher_a, researcher_b],
)

# Sequential wraps: parallel stage → synthesizer
root_agent = SequentialAgent(
    name="ResearchWorkflow",
    sub_agents=[parallel_research, synthesizer],
)

Overwriting workflow_agent/agent.py


In [94]:
# Run the workflow programmatically using Runner
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai.types import Content, Part
from workflow_agent.agent import root_agent

# Create the session service and runner
session_service = InMemorySessionService()
runner = Runner(
    agent=root_agent,
    app_name="research_app",
    session_service=session_service,
)

# Create a session
session = await session_service.create_session(
    app_name="research_app",
    user_id="tutorial_user",
)

# Send a message and collect the response
message = Content(parts=[Part(text="Research clean energy and EVs.")], role="user")

response_text = ""
async for event in runner.run_async(
    user_id="tutorial_user",
    session_id=session.id,
    new_message=message,
):
    if event.is_final_response() and event.content and event.content.parts:
        response_text = event.content.parts[0].text

print("=== Synthesized Result ===")
print(response_text)

=== Synthesized Result ===
The rapid evolution towards sustainable transportation is underpinned by significant advancements in both clean energy generation and electric vehicle technology. Renewable energy, particularly solar photovoltaics and wind power, has seen unprecedented cost reductions and efficiency gains, making it globally competitive, a trend further supported by breakthroughs in grid-scale energy storage and smarter grids capable of integrating intermittent sources. Concurrently, electric vehicles have made remarkable progress, with battery technology offering improved range, faster charging, and lower manufacturing costs, complemented by an expanding charging infrastructure and enhanced performance. This synergy, where a more robust and renewable-powered grid can efficiently fuel an increasingly adopted fleet of advanced EVs, positions electric transportation as a dominant future solution, deeply intertwined with the broader clean energy transition.


### When to Use Which Agent Type

| Agent Type | Use When... | Example |
|------------|------------|--------|
| **LlmAgent** | You need LLM reasoning, tool use, or dynamic decisions | Chatbots, Q&A, tool-calling agents |
| **SequentialAgent** | Steps must happen in order, each building on the previous | Write → Review → Refactor pipeline |
| **ParallelAgent** | Tasks are independent and can run concurrently | Multi-source research, parallel API calls |
| **LoopAgent** | Iterative refinement until quality threshold is met | Draft → Critique → Revise loops |

---

## 6. Tools

Tools give agents the ability to take actions and access external data. ADK has one of the most extensive tool ecosystems of any agent framework.

![Tools in ADK](images/image(3).png)

| Category | Tool | Description |
|----------|------|-------------|
| **Function Tools** | `FunctionTool` | Auto schema from docstrings + type hints |
| | `LongRunningFunctionTool` | Async ops & human-in-the-loop |
| **Agent-as-Tool** | `AgentTool` | Wrap any agent as a callable tool; cross-framework interop |
| **Protocol Tools** | `MCPToolset` | Connect to MCP servers (Stdio & SSE transport) |
| | `OpenAPIToolset` | Auto-generate tools from OpenAPI specs |
| **Built-in Google** | `google_search` | Gemini 2+ only |
| | `BigQueryToolset` | 7 built-in tools: SQL, forecast, insights |
| | `VertexAiSearchTool` | Private data stores |
| | `BuiltInCodeExecutor` | Sandboxed Python |
| **Runtime Access** | `ToolContext` | `state`, `actions`, `artifacts`, `memory`, `auth credentials` |

### Function Tools

Any regular Python function can become a tool. ADK inspects the function's **name**, **docstring**, **type hints**, and **defaults** to auto-generate a schema for the LLM.

Rules:
- Parameters without defaults are **required**; with defaults are **optional**
- Return a `dict` with a `"status"` key for best results
- Docstrings and type hints are critical — they tell the LLM what the tool does and how to call it
- Add `tool_context: ToolContext` as a parameter to access runtime state, actions, artifacts, and memory

In [95]:
# Custom function tool example
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai.types import Content, Part


def get_weather(city: str) -> dict:
    """Retrieves the current weather report for a specified city.

    Args:
        city (str): The city name to get weather for.
    """
    # Simulated weather data
    weather_data = {
        "new york": {"temp": "22°C", "condition": "Sunny"},
        "london": {"temp": "15°C", "condition": "Cloudy"},
        "tokyo": {"temp": "28°C", "condition": "Humid"},
    }
    city_lower = city.lower()
    if city_lower in weather_data:
        data = weather_data[city_lower]
        return {
            "status": "success",
            "report": f"The weather in {city} is {data['condition']}, {data['temp']}.",
        }
    return {
        "status": "error",
        "error_message": f"Weather data for '{city}' is not available.",
    }


# Create an agent with the tool
weather_agent = Agent(
    name="weather_agent",
    model="gemini-2.5-flash",
    description="Agent that answers weather questions.",
    instruction="You are a weather assistant. Use the get_weather tool to answer questions about weather.",
    tools=[get_weather],
)

# Run it
session_service = InMemorySessionService()
runner = Runner(agent=weather_agent, app_name="weather_app", session_service=session_service)
session = await session_service.create_session(app_name="weather_app", user_id="tutorial_user")

message = Content(parts=[Part(text="What's the weather like in Tokyo?")], role="user")

async for event in runner.run_async(
    user_id="tutorial_user",
    session_id=session.id,
    new_message=message,
):
    if event.is_final_response() and event.content and event.content.parts:
        print(event.content.parts[0].text)

The weather in Tokyo is Humid, 28°C.


### Built-in Tools

ADK ships with several built-in tools for Google Cloud services:

**Google Search** (`google_search`):
- Import: `from google.adk.tools import google_search`
- Requires Gemini 2+ models
- **Single-tool limitation**: Google Search cannot be combined with other tools in a single agent. Workaround: use a sub-agent pattern where a dedicated search agent handles Google Search, and the parent agent delegates to it.

**BigQuery** (`BigQueryToolset`):
- Import: `from google.adk.tools import BigQueryToolset`
- Provides 7 built-in tools: SQL queries, forecasting, insights, and more
- Operates on your GCP project's BigQuery datasets

In [96]:
# Set up Search Agent directory
import os
os.makedirs("search_agent", exist_ok=True)

In [97]:
%%writefile search_agent/__init__.py
from . import agent

Overwriting search_agent/__init__.py


In [98]:
%%writefile search_agent/.env
GOOGLE_GENAI_USE_VERTEXAI=TRUE
GOOGLE_CLOUD_PROJECT=agentspace-testing-471714
GOOGLE_CLOUD_LOCATION=us-central1

Overwriting search_agent/.env


In [99]:
%%writefile search_agent/agent.py
from google.adk.agents import Agent
from google.adk.tools import google_search

root_agent = Agent(
    name="search_agent",
    model="gemini-2.5-flash",
    description="Searches the web for information.",
    instruction="You are a web search specialist. Use Google Search to find information and provide concise answers.",
    tools=[google_search],
)

Overwriting search_agent/agent.py


In [100]:
# Set up BigQuery Agent directory
import os
os.makedirs("bq_agent", exist_ok=True)

In [101]:
%%writefile bq_agent/__init__.py
from . import agent

Overwriting bq_agent/__init__.py


In [102]:
%%writefile bq_agent/.env
GOOGLE_GENAI_USE_VERTEXAI=TRUE
GOOGLE_CLOUD_PROJECT=agentspace-testing-471714
GOOGLE_CLOUD_LOCATION=us-central1

Overwriting bq_agent/.env


In [103]:
%%writefile bq_agent/agent.py
from google.adk.agents import Agent
from google.adk.tools.bigquery.bigquery_toolset import BigQueryToolset

root_agent = Agent(
    name="bigquery_agent",
    model="gemini-2.5-flash",
    description="Queries BigQuery datasets.",
    instruction=(
        "You are a data analyst assistant with access to BigQuery.\n"
        "The dataset `agentspace-testing-471714.adk_learning.shakespeare` contains "
        "Shakespeare's works with columns: word, word_count, corpus, corpus_date.\n"
        "Always explain your findings clearly."
    ),
    tools=[BigQueryToolset()],
)

Overwriting bq_agent/agent.py


In [104]:
# Run Search and BigQuery agents programmatically
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai.types import Content, Part
from search_agent.agent import root_agent as search_agent
from bq_agent.agent import root_agent as bigquery_agent

print("=== Google Search ===")
session_service = InMemorySessionService()
runner = Runner(agent=search_agent, app_name="search_app", session_service=session_service)
session = await session_service.create_session(app_name="search_app", user_id="tutorial_user")

message = Content(parts=[Part(text="What is Shakespeare's most famous play?")], role="user")
async for event in runner.run_async(
    user_id="tutorial_user", session_id=session.id, new_message=message,
):
    if event.is_final_response() and event.content and event.content.parts:
        print(event.content.parts[0].text)

# Demo 2: BigQuery
print("\n=== BigQuery ===")
session_service = InMemorySessionService()
runner = Runner(agent=bigquery_agent, app_name="bq_app", session_service=session_service)
session = await session_service.create_session(app_name="bq_app", user_id="tutorial_user")

message = Content(
    parts=[Part(text=(
        "Count how many distinct works (corpus values) are in the "
        "agentspace-testing-471714.adk_learning.shakespeare dataset."
    ))],
    role="user",
)
async for event in runner.run_async(
    user_id="tutorial_user", session_id=session.id, new_message=message,
):
    if event.is_final_response() and event.content and event.content.parts:
        print(event.content.parts[0].text)

=== Google Search ===


While there isn't a single definitive answer to Shakespeare's "most famous" play, *Hamlet* is consistently cited as his most popular and critically acclaimed work. It has engrossed audiences and readers since its first recorded production and is known for challenging even the most gifted actors. The play is recognized for its depth and complexity, following Prince Hamlet's quest for vengeance against his father's murderer. Many common expressions in modern language also originate from *Hamlet*.

Other highly famous and frequently performed plays include *Romeo and Juliet*, celebrated for its lyrical poetry, and *Macbeth*, a prominent tragedy in theatre history. *A Midsummer Night's Dream* is also noted as a very popular and frequently performed play, especially in the United States and abroad, due to its accessible comedic elements.

=== BigQuery ===
There are 42 distinct works (corpus values) in the `bigquery-public-data.samples.shakespeare` dataset.


### Other Tool Types

| Tool Type | Class | Use Case |
|-----------|-------|----------|
| **Function Tool** | `FunctionTool` | Any Python function — auto-generates schema from docstring + type hints |
| **Long-Running** | `LongRunningFunctionTool` | Async operations, human-in-the-loop approval workflows |
| **Agent-as-Tool** | `AgentTool` | Wrap any agent as a callable tool — enables cross-framework interop |
| **MCP Tools** | `McpToolset` | Connect to Model Context Protocol servers (Stdio or SSE transport) |
| **OpenAPI Tools** | `OpenAPIToolset` | Auto-generate tools from an OpenAPI spec (JSON or YAML) |

### ToolContext

Any tool function can accept a `tool_context: ToolContext` parameter for runtime access:

| Property | Description |
|----------|-------------|
| `tool_context.state` | Read/write session state (supports prefix scoping) |
| `tool_context.actions` | Control flow — `escalate`, `transfer_to_agent`, `skip_summarization` |
| `tool_context.save_artifact(name, part)` | Save binary data (files, images) |
| `tool_context.load_artifact(name)` | Load previously saved artifacts |
| `tool_context.search_memory(query)` | Query the memory service |
| `tool_context.function_call_id` | ID of the current function call |

```python
from google.adk.tools import ToolContext

def my_tool(query: str, tool_context: ToolContext) -> dict:
    tool_context.state["temp:last_query"] = query
    tool_context.state["user:query_count"] = tool_context.state.get("user:query_count", 0) + 1
    return {"status": "success", "result": "..."}
```

---

## 7. Callbacks

Callbacks hook into an agent's execution at predefined points. They let you observe, modify, or override behavior — without changing the agent's core logic.

### Callback Lifecycle

The execution flow follows this sequence:

| Step | Event | Callback | Skip / Override |
|------|-------|----------|-----------------|
| 1 | User message received | `before_agent_callback` | Return `Content` to skip agent entirely |
| 2 | Prompt sent to LLM | `before_model_callback` | Return `LlmResponse` to skip LLM call |
| 3 | LLM response received | `after_model_callback` | Return `LlmResponse` to replace response |
| 4 | Tool call initiated | `before_tool_callback` | Return `dict` to skip tool execution |
| 5 | Tool result returned | `after_tool_callback` | Return `dict` to replace tool result |
| 6 | Agent response finalized | `after_agent_callback` | Return `Content` to replace final output |

**Pattern:** Return `None` to proceed normally, or return a value to **override** the default behavior.

### The Six Callbacks

| Callback | Parameters | Return `None` | Return a Value |
|----------|-----------|--------------|----------------|
| `before_agent_callback` | `CallbackContext` | Agent runs normally | `Content` — skip the agent entirely |
| `after_agent_callback` | `CallbackContext` | Use agent's output | `Content` — replace the output |
| `before_model_callback` | `CallbackContext`, `LlmRequest` | LLM call proceeds | `LlmResponse` — skip the LLM call |
| `after_model_callback` | `CallbackContext`, `LlmResponse` | Use LLM response | `LlmResponse` — replace the response |
| `before_tool_callback` | `BaseTool`, `args: dict`, `ToolContext` | Tool runs normally | `dict` — skip the tool |
| `after_tool_callback` | `BaseTool`, `args: dict`, `ToolContext`, `tool_response: dict` | Use tool result | `dict` — replace the result |

The pattern is simple: return `None` to proceed normally, or return a value to **override** the default behavior.

In [105]:
# Guardrail callback: before_model_callback blocks forbidden keywords
import os
os.makedirs("callback_agent", exist_ok=True)

In [106]:
%%writefile callback_agent/__init__.py
from . import agent

Overwriting callback_agent/__init__.py


In [107]:
%%writefile callback_agent/.env
GOOGLE_GENAI_USE_VERTEXAI=TRUE
GOOGLE_CLOUD_PROJECT=agentspace-testing-471714
GOOGLE_CLOUD_LOCATION=us-central1

Overwriting callback_agent/.env


In [108]:
%%writefile callback_agent/agent.py
from typing import Optional
from google.adk.agents import Agent
from google.adk.agents.callback_context import CallbackContext
from google.adk.models import LlmRequest, LlmResponse
from google.adk.tools.base_tool import BaseTool
from google.adk.tools import ToolContext
from google.genai import types

FORBIDDEN_WORDS = ["hack", "exploit", "bypass"]


def guardrail_before_model(
    callback_context: CallbackContext,
    llm_request: LlmRequest,
) -> Optional[LlmResponse]:
    """Block requests containing forbidden keywords."""
    # Safely extract text from the last user message
    user_text = ""
    for content in reversed(llm_request.contents):
        if content.role == "user":
            for part in content.parts:
                if part.text:
                    user_text = part.text.lower()
                    break
            if user_text:
                break
    for word in FORBIDDEN_WORDS:
        if word in user_text:
            print(f"[GUARDRAIL] Blocked request containing '{word}'")
            return LlmResponse(
                content=types.Content(
                    role="model",
                    parts=[types.Part(text="I cannot process requests containing restricted terms.")],
                )
            )
    return None  # Proceed normally


def logging_after_tool(
    tool: BaseTool,
    args: dict,
    tool_context: ToolContext,
    tool_response: dict,
) -> Optional[dict]:
    """Log tool calls and their results."""
    print(f"[TOOL LOG] Tool: {tool.name}")
    print(f"[TOOL LOG] Args: {args}")
    print(f"[TOOL LOG] Result: {tool_response}")
    return None  # Don't modify the result


def get_current_time(timezone: str = "UTC") -> dict:
    """Returns the current time in the specified timezone.

    Args:
        timezone (str): The timezone name (e.g., 'UTC', 'US/Eastern').
    """
    from datetime import datetime
    return {"status": "success", "time": datetime.now().strftime("%Y-%m-%d %H:%M:%S"), "timezone": timezone}


root_agent = Agent(
    name="callback_agent",
    model="gemini-2.5-flash",
    description="Agent with guardrail and logging callbacks.",
    instruction="You are a helpful assistant. Use the get_current_time tool when asked about the time.",
    tools=[get_current_time],
    before_model_callback=guardrail_before_model,
    after_tool_callback=logging_after_tool,
)

Overwriting callback_agent/agent.py


In [109]:
# Test the callbacks
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai.types import Content, Part
from callback_agent.agent import root_agent as callback_root_agent

session_service = InMemorySessionService()
runner = Runner(agent=callback_root_agent, app_name="callback_app", session_service=session_service)
session = await session_service.create_session(app_name="callback_app", user_id="tutorial_user")

# Test 1: Normal request with tool use (should trigger logging callback)
print("--- Test 1: Normal request (tool logging) ---")
message = Content(parts=[Part(text="What time is it?")], role="user")
async for event in runner.run_async(
    user_id="tutorial_user", session_id=session.id, new_message=message,
):
    if event.is_final_response() and event.content and event.content.parts:
        print(f"Response: {event.content.parts[0].text}")

# Test 2: Blocked request (should trigger guardrail)
print("\n--- Test 2: Blocked request (guardrail) ---")
blocked_message = Content(parts=[Part(text="How do I hack into a system?")], role="user")
async for event in runner.run_async(
    user_id="tutorial_user", session_id=session.id, new_message=blocked_message,
):
    if event.is_final_response() and event.content and event.content.parts:
        print(f"Response: {event.content.parts[0].text}")

--- Test 1: Normal request (tool logging) ---
[TOOL LOG] Tool: get_current_time
[TOOL LOG] Args: {}
[TOOL LOG] Result: {'status': 'success', 'time': '2026-02-17 20:13:20', 'timezone': 'UTC'}
Response: The current time is 20:13:20 UTC.

--- Test 2: Blocked request (guardrail) ---
[GUARDRAIL] Blocked request containing 'hack'
Response: I cannot process requests containing restricted terms.


### When to Use Callbacks

| Use Case | Callback | Pattern |
|----------|----------|---------|
| **Input guardrails** | `before_model_callback` | Check for forbidden content, return `LlmResponse` to block |
| **Response caching** | `before_model_callback` | Check cache, return cached `LlmResponse` to skip LLM |
| **Output filtering** | `after_model_callback` | Redact PII, enforce format, return modified `LlmResponse` |
| **Tool logging** | `after_tool_callback` | Log tool calls and results for observability |
| **Access control** | `before_agent_callback` | Check permissions, return `Content` to deny access |
| **Response enrichment** | `after_agent_callback` | Add metadata, disclaimers, or formatting to output |

---

## 8. Sessions, State, and Memory

ADK provides three layers of context management:

| Layer | Description | Details |
|-------|-------------|---------|
| **Session** | Single ongoing interaction | Fields: `id`, `app_name`, `user_id`, `events`, `state`, `last_update_time` |
| **State** (prefixes) | Key-value store within a session | `(none)` = current session, `temp:` = current invocation only, `user:` = cross-session per user, `app:` = global |
| **Memory** | Cross-session knowledge store | Persists beyond sessions; supports keyword matching and semantic search |

**Session backends:** `InMemorySessionService` (local testing) · `DatabaseSessionService` (SQLite/PostgreSQL) · `VertexAiSessionService` (managed production)

**Memory backends:** `InMemoryMemoryService` (keyword matching, prototyping) · `VertexAiMemoryBankService` (semantic search, production)

### Sessions

A **Session** represents a single ongoing interaction (conversation). Key fields:

| Field | Description |
|-------|-------------|
| `id` | Unique conversation identifier |
| `app_name` | Which agent application owns this session |
| `user_id` | Links to a particular user |
| `events` | Chronological sequence of all interactions |
| `state` | Key-value store for conversation data |

**SessionService backends:**

| Service | Backend | Use Case |
|---------|---------|----------|
| `InMemorySessionService` | In-memory | Local testing (lost on restart) |
| `DatabaseSessionService` | SQLite, PostgreSQL, MySQL | Self-hosted production |
| `VertexAiSessionService` | Vertex AI Agent Engine | Managed GCP production |

In [110]:
# Session demo: create a session with initial state, run agent, inspect results
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai.types import Content, Part

session_agent = Agent(
    name="session_agent",
    model="gemini-2.5-flash",
    instruction="You are a helpful assistant. The user's name is {user_name}. Greet them by name.",
)

session_service = InMemorySessionService()
runner = Runner(agent=session_agent, app_name="session_app", session_service=session_service)

# Create session with initial state
session = await session_service.create_session(
    app_name="session_app",
    user_id="tutorial_user",
    state={"user_name": "Alice"},  # Initial state
)

message = Content(parts=[Part(text="Hi there!")], role="user")
async for event in runner.run_async(
    user_id="tutorial_user", session_id=session.id, new_message=message,
):
    if event.is_final_response() and event.content and event.content.parts:
        print(f"Agent: {event.content.parts[0].text}")

# Inspect session state and events
updated_session = await session_service.get_session(
    app_name="session_app", user_id="tutorial_user", session_id=session.id,
)
print(f"\nSession ID: {updated_session.id}")
print(f"State: {dict(updated_session.state)}")
print(f"Number of events: {len(updated_session.events)}")

Agent: Hello Alice! How can I help you today?

Session ID: cf54f176-26b7-4003-8a82-503f52ae7c08
State: {'user_name': 'Alice'}
Number of events: 2


### State

`session.state` is a key-value dictionary with **prefix-based scoping**:

| Prefix | Scope | Persistence |
|--------|-------|-------------|
| *(none)* | Current session only | Session lifetime |
| `temp:` | Current invocation only | **Never** persisted |
| `user:` | All sessions for that user | With DB/Vertex services |
| `app:` | All users and sessions | With DB/Vertex services |

**Three ways to write state:**

1. **`output_key`** — Agent's response auto-saved: `Agent(output_key="last_response", ...)`
2. **`ToolContext.state`** — Recommended in tool functions: `tool_context.state["key"] = value`
3. **`EventActions.state_delta`** — Low-level, via event actions

In [111]:
# State demo: tool writes to different prefixes, output_key auto-save
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import ToolContext
from google.genai.types import Content, Part


def track_query(query: str, tool_context: ToolContext) -> dict:
    """Tracks user queries and updates state with different prefixes.

    Args:
        query (str): The user's search query.
    """
    # temp: prefix — only lasts this invocation
    tool_context.state["temp:current_query"] = query

    # user: prefix — persists across sessions (with DB/Vertex services)
    count = tool_context.state.get("user:total_queries", 0)
    tool_context.state["user:total_queries"] = count + 1

    # No prefix — persists within this session
    tool_context.state["last_query"] = query

    return {"status": "success", "message": f"Tracked query: {query}"}


state_agent = Agent(
    name="state_agent",
    model="gemini-2.5-flash",
    instruction="You track queries. When the user asks something, use track_query first, then answer.",
    tools=[track_query],
    output_key="last_response",  # Auto-save response to state
)

session_service = InMemorySessionService()
runner = Runner(agent=state_agent, app_name="state_app", session_service=session_service)
session = await session_service.create_session(app_name="state_app", user_id="tutorial_user")

message = Content(parts=[Part(text="Track a query about 'Python async patterns'")], role="user")
async for event in runner.run_async(
    user_id="tutorial_user", session_id=session.id, new_message=message,
):
    if event.is_final_response() and event.content and event.content.parts:
        print(f"Agent: {event.content.parts[0].text}")

# Inspect state after
updated_session = await session_service.get_session(
    app_name="state_app", user_id="tutorial_user", session_id=session.id,
)
print("\n--- Session State ---")
for key, value in sorted(updated_session.state.items()):
    print(f"  {key}: {value}")

Agent: I have tracked your query about 'Python async patterns'. Is there anything else I can help you with regarding this topic?

--- Session State ---
  last_query: Python async patterns
  last_response: I have tracked your query about 'Python async patterns'. Is there anything else I can help you with regarding this topic?
  user:total_queries: 1


### Memory

**Memory** is long-term knowledge that persists **across sessions**. While State is per-session, Memory stores extracted facts that any future session can retrieve.

| Service | Backend | Features |
|---------|---------|----------|
| `InMemoryMemoryService` | In-memory | Basic keyword matching, for prototyping |
| `VertexAiMemoryBankService` | Vertex AI | LLM-powered extraction, semantic search |

**Workflow:**
1. User interacts with agent → session events captured
2. Call `memory_service.add_session_to_memory(session)` → extracts and stores key facts
3. In a later session, agent querys with `search_memory(query)` → retrieves relevant memories

**Built-in memory tools:**
- `PreloadMemoryTool` — automatically retrieves relevant memories at the start of each turn
- `LoadMemoryTool` — retrieves on-demand when the agent decides to search

In [112]:
# Memory demo: Session A stores info → add to memory → Session B retrieves it
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.memory import InMemoryMemoryService
from google.adk.tools.preload_memory_tool import PreloadMemoryTool
from google.genai.types import Content, Part

memory_service = InMemoryMemoryService()
session_service = InMemorySessionService()

# Agent with memory retrieval
memory_agent = Agent(
    name="memory_agent",
    model="gemini-2.5-flash",
    instruction=(
        "You are a helpful assistant with memory. "
        "Use information from memory to provide personalized responses. "
        "If memory has relevant info, mention it."
    ),
    tools=[PreloadMemoryTool()],
)

runner = Runner(
    agent=memory_agent,
    app_name="memory_app",
    session_service=session_service,
    memory_service=memory_service,
)

# --- Session A: Store information ---
print("=== Session A: Storing information ===")
session_a = await session_service.create_session(
    app_name="memory_app", user_id="tutorial_user",
)

msg_a = Content(parts=[Part(text="My favorite programming language is Python and I work at Acme Corp.")], role="user")
async for event in runner.run_async(
    user_id="tutorial_user", session_id=session_a.id, new_message=msg_a,
):
    if event.is_final_response() and event.content and event.content.parts:
        print(f"Agent: {event.content.parts[0].text}")

# Add session A to memory
updated_session_a = await session_service.get_session(
    app_name="memory_app", user_id="tutorial_user", session_id=session_a.id,
)
await memory_service.add_session_to_memory(updated_session_a)
print("\n[Memory updated with Session A data]")


# --- Session B: Retrieve from memory ---
print("\n=== Session B: Retrieving from memory ===")
session_b = await session_service.create_session(
    app_name="memory_app", user_id="tutorial_user",
)

msg_b = Content(parts=[Part(text="What do you know about me?")], role="user")
async for event in runner.run_async(
    user_id="tutorial_user", session_id=session_b.id, new_message=msg_b,
):
    if event.is_final_response() and event.content and event.content.parts:
        print(f"Agent: {event.content.parts[0].text}")

=== Session A: Storing information ===
Agent: Okay, I've noted that your favorite programming language is Python and you work at Acme Corp. Thanks for sharing!

[Memory updated with Session A data]

=== Session B: Retrieving from memory ===
Agent: Based on our previous conversations, I know two things about you:

1.  Your favorite programming language is Python.
2.  You work at Acme Corp.


---

## 9. Clean Up

Remove the agent directories created during this tutorial.

In [113]:
#import shutil

## Stop adk web server if still running
#try:
#    if adk_web_process.poll() is None:
#        adk_web_process.terminate()
#        adk_web_process.wait(timeout=5)
#        print("Stopped adk web server.")
#except NameError:
#    pass  # adk_web_process was never created

#for agent_dir in ["hello_agent", "workflow_agent", "search_agent", "bq_agent", "callback_agent"]:
#    if os.path.exists(agent_dir):
#        shutil.rmtree(agent_dir)
#        print(f"Removed {agent_dir}/")
#    else:
#        print(f"{agent_dir}/ not found (already removed)")

### What's Next?

![Key Takeaways](images/image(2).png)

You've covered the core of ADK. Here are suggested next steps:

- **Deploy your agent**: Try `adk deploy cloud_run` to ship to Google Cloud Run
- **Add evaluation**: Create `.test.json` files and run `adk eval` to measure quality
- **Explore MCP tools**: Connect to external services via the Model Context Protocol
- **Use Vertex AI Agent Engine**: Deploy to the fully managed service for production
- **Try multi-model agents**: Use `LiteLlm` to combine Gemini with Claude, GPT-4, or local models

**Resources:**

- [ADK Documentation](https://google.github.io/adk-docs/)
- [ADK GitHub (Python)](https://github.com/google/adk-python)
- [ADK Quickstart](https://google.github.io/adk-docs/get-started/quickstart/)
- [Vertex AI Agent Engine](https://cloud.google.com/vertex-ai/generative-ai/docs/agent-engine/overview)

### Credits

- **Google Agent Development Kit** — [google.github.io/adk-docs](https://google.github.io/adk-docs/)
- **Gemini API** — [ai.google.dev](https://ai.google.dev/)
- Notebook generated with assistance from Claude